# Status: Working

 TO DO:
 - Handle Errors Approriately 

In [216]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ActionChains
import os
from selenium.common.exceptions import ElementClickInterceptedException

In [217]:
SAVE_IMG = os.getcwd()+"/image.png"

In [218]:
EXPECTED_NASDAQ_FILTER_VALUE = "exchange~NASDAQ"
STOCK_SCREENER_URL = "https://www.nasdaq.com/market-activity/stocks/screener"

In [219]:
options = webdriver.ChromeOptions()
options.page_load_strategy= "eager"
assert options.capabilities['browserName'] == 'chrome'
preferences = {"download.default_directory":os.getcwd()}
options.add_experimental_option("prefs",preferences)
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(7)
driver.get(STOCK_SCREENER_URL)


In [220]:
shadowNASDAQFilterParentElement = driver.find_element(By.ID,"exchange-radio-group-id")
shadowRootNASDAQFilter = driver.execute_script("return arguments[0].shadowRoot", shadowNASDAQFilterParentElement)
nasdaqRadioButton= shadowRootNASDAQFilter.find_element(By.ID, "option-NASDAQ")
nasdaqRadioButton.click()

In [221]:
filterParentElement = driver.find_element(By.CLASS_NAME,"jupiter22-c-filter-button-container")
applyFilterElement = filterParentElement.find_element(By.CLASS_NAME ,"jupiter22-c-button__apply")
try:
    applyFilterElement.click()    
except ElementClickInterceptedException:
    footer = driver.find_element(By.TAG_NAME, "footer")
    delta_y = int(footer.rect['y'])
    ActionChains(driver).scroll_to_element(applyFilterElement).pause(1).scroll_by_amount(0, delta_y).pause(1).scroll_to_element(applyFilterElement).perform()
    applyFilterElement.click()

In [222]:
filtersAppliedParentElement = driver.find_element(By.CLASS_NAME,"jupiter22-c-table-filters__filters")
foundFilterElements = filtersAppliedParentElement.find_elements(By.CLASS_NAME,"jupiter22-c-table-filters__chip")
foundFilterAttributes = list(map(lambda x: x.get_attribute("data-filter"),foundFilterElements))
if ((foundFilterAttributes.__len__() ==1) & (foundFilterAttributes[0]==EXPECTED_NASDAQ_FILTER_VALUE)):
    pass
else:
    raise Exception("FILTER NOT FOUND: ",EXPECTED_NASDAQ_FILTER_VALUE)

In [224]:
downloadCSVElement = driver.find_element(By.CLASS_NAME,"jupiter22-c-table__download-csv")
downloadCSVElement.click()

In [225]:
wait = WebDriverWait(driver, timeout=14, poll_frequency=1)
isSuccess = wait.until(lambda done : 1==sum(map(lambda x : ".csv" in x,os.listdir(os.getcwd()))),"File not found in Expected Directory")
print("Download Successful!") if isSuccess else print("Download Failed!")

Download Successful!


In [226]:
driver.close()